# mnb parameter tuning (Multinomial Naieve Bayes)

In [97]:
from __future__ import division
import etl


In [4]:
s = etl.Server()
data_set = etl.load_dir('data/ml_challenge')

In [5]:
orig_X_train, orig_Y_train, orig_train_targets = data_set['train']['binary_data'], data_set['train']['answers'], data_set['train']['targets']
orig_X_dev, orig_Y_dev, orig_dev_targets = data_set['dev']['binary_data'], data_set['dev']['answers'], data_set['dev']['targets']
orig_X_test, orig_Y_test, orig_test_targets = data_set['test']['binary_data'], data_set['test']['answers'], data_set['test']['targets']

print('orig_X_train[0:4]', '\n', orig_X_train[0:4])
print('orig_Y_train[0:4]', '\n', orig_Y_train[0:4])
print('orig_train_targets[0:4]', '\n', orig_train_targets[0:4])

('orig_X_train[0:4]', '\n', [u'AEjnPgBCuQAAAAAmOQAAAAAYAyA5AAAAACA5AAAAAHQMcgFTAmcwIDkAAAAAcAFChkxGCAAKQAABAgAAAYAERg==', u'kPS4A1EAAKC7MJkFmVi5OBYE/bEAAEEAACgLwCAAWAShAAAgIATAIABIBCkL8gYAsQAAqQWpBPCgdMAgAKkLwA==', u'AAAAAAAAAAAAAP//QiQAAAM8IABirAAAAjwgAEKMAAAAAPP/QQRAIAIAAAACPAQAABAcAECkAAADPBwAYqQAAA==', u'AEOsAQCEJPD/gBAAAAAAEwAAEAEABCQhEAAAAgBDKP7/YBQBAEIkAAACPAQAABAQAECsAQBCJBAAYqwAAAI8EA=='])
('orig_Y_train[0:4]', '\n', [u'm68k', u'xtensa', u'mipsel', u'mipsel'])
('orig_train_targets[0:4]', '\n', [[u'alphaev56', u'arm', u'm68k', u'mipsel', u's390', u'x86_64'], [u'avr', u'mips', u'powerpc', u'sh4', u'x86_64', u'xtensa'], [u'm68k', u'mips', u'mipsel', u's390', u'sparc', u'xtensa'], [u'arm', u'm68k', u'mipsel', u'powerpc', u's390', u'sparc']])


In [6]:
hex_X_train, train_misfits = etl.hex_data(orig_X_train)
hex_X_dev, dev_misfits = etl.hex_data(orig_X_dev)
hex_X_test, test_misfits = etl.hex_data(orig_X_test)


In [7]:
hexdoc_X_train = map("".join, hex_X_train)
hexdoc_X_test =  map("".join, hex_X_test)
hexdoc_X_dev = map("".join, hex_X_dev)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import numpy as np


In [9]:
def guess_from_target(probs, allowed_Y, supported_architectures):
    """
    Improve our chances by taking the max over the possible targets (6 instead of 12)
    probs: numerical array of shape (m, n_classes)
    targets: ones-hot array of shape (m, n_classes)
    
    returns: (m, 1) of the most likely ISA arch names after discards or
             (m, n_classes) one-hot representation of best guess
    """
    return map(supported_architectures.__getitem__, np.argmax(probs*allowed_Y, axis=1))

In [78]:
def describe_results(predictions, orig_Y, param_str, error_type='train'):
    wrong = []
    for i in range(len(predictions)):
        if predictions[i] != orig_Y[i]:
            wrong.append([predictions[i], orig_Y[i]])
    if error_type == 'train':
        print(param_str)
    print('{} error: {}'.format(error_type, len(wrong)/len(predictions)))
    return wrong, len(wrong)/len(predictions)

In [79]:
params = {}
params['alpha'] = [0.0001, 0.001, 0.1, .99, 1.0]
params['max_ngram_range'] = [6, 8]
params['smooth_idf'] = [True, False]
params['norm'] = ['l2', None]
params['sublinear_tf'] = [True, False]
params['min_df'] = [1, 3]
params['max_df'] = [.5, .9]


In [91]:
def search_hyperparams():
    df = pd.DataFrame(params.keys() + ['train_error', 'dev_error'])
    for alpha in params['alpha']:
        wrong_train = []
        wrong_dev = []
        for max_ngram_range in params['max_ngram_range']:
            for smooth_idf in params['smooth_idf']:
                for norm in params['norm']:
                    for sublinear_tf in params['sublinear_tf']:
                        for min_df in params['min_df']:
                            for max_df in params['max_df']:
                                vec_opts = {
                                    "ngram_range": (1, max_ngram_range),  # allow n-grams of 1-4 words in length (32-bits)
                                    "analyzer": "word",     # analyze hex words
                                    "token_pattern": "..",  # treat two characters as a word (e.g. 4b)
                                    "min_df": min_df,          # for demo purposes, be very selective about features
                                    "max_df": max_df
                                }
                                v = CountVectorizer(**vec_opts)
                                X_cv = v.fit_transform(hexdoc_X_train)

                                idf_opts = {"use_idf": True}
                                idf = TfidfTransformer(**idf_opts)

                                # perform the idf transform
                                X_idf = idf.fit_transform(X_cv)

                                mnbClassifier = MultinomialNB(alpha=alpha)

                                mnb_model = mnbClassifier.fit(X_idf, np.array(orig_Y_train))

                                Y_train, allowed_Y_train = etl.class_to_ones_hot(orig_Y_train, orig_train_targets, mnb_model.classes_.tolist())
                                Y_dev, allowed_Y_dev = etl.class_to_ones_hot(orig_Y_dev, orig_dev_targets, mnb_model.classes_.tolist())
                                Y_test, allowed_Y_test = etl.class_to_ones_hot(orig_Y_test, orig_test_targets, mnb_model.classes_.tolist())

                                probs_train = mnb_model.predict_proba(X_idf)

                                predictions = guess_from_target(probs_train, allowed_Y_train, mnb_model.classes_.tolist())

                                param_str = "alpha={}, max_ngram_range={}, smooth_idf={}, norm={}, sublinear_tf={}, min_df={}, max_df={}".format(
                                alpha, max_ngram_range, smooth_idf, norm, sublinear_tf, min_df, max_df)
                                mismatches, train_error = describe_results(predictions, orig_Y_train, param_str)
                                wrong_train.append(mismatches)


                                vec_opts.update({'vocabulary': v.vocabulary_})
                                v_dev = CountVectorizer(**vec_opts)
                                X_cv_dev = v_dev.transform(hexdoc_X_dev)
                                X_idf_dev = idf.transform(X_cv_dev)
                                probs_dev = mnb_model.predict_proba(X_idf_dev)

                                predictions_dev = guess_from_target(probs_dev, allowed_Y_dev, mnb_model.classes_.tolist())
                                mismatches, dev_error = describe_results(predictions_dev, orig_Y_dev, param_str, error_type='dev')
                                wrong_dev.append(mismatches)
                                
                                df = df.append({'alpha': alpha, 'max_ngram_range': max_ngram_range, 
                                                'smooth_idf': smooth_idf, 'norm': norm, 
                                          'sublinear_tf': sublinear_tf, 'min_df': min_df, 
                                          'max_df': max_df, 'train_error': train_error, 
                                          'dev_error': dev_error}, ignore_index=True)
    return wrong_train, wrong_dev, df

In [92]:
wrong_train, wrong_dev, df_mnb = search_hyperparams()

alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2,

dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_n

dev error: 0.355113636364%
alpha=0.001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0590479651163%
dev error: 0.355113636364%
alpha=0.001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0635901162791%
dev error: 0.355113636364%
alpha=0.001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.001, max_ngram_ra

dev error: 0.390625%
alpha=0.1, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.258902616279%
dev error: 0.390625%
alpha=0.1, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.254360465116%
dev error: 0.390625%
alpha=0.1, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.0181686046512%
dev error: 0.461647727273%
alpha=0.1, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.0181686046512%
dev error: 0.461647727273%
alpha=0.1, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.240734011628%
dev error: 0.390625%
alpha=0.1, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.240734011628%
dev error: 0.390625%
alpha=0.1, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_d

dev error: 0.568181818182%
alpha=0.99, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.27707122093%
dev error: 0.532670454545%
alpha=0.99, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.286155523256%
dev error: 0.532670454545%
alpha=0.99, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.458757267442%
dev error: 0.568181818182%
alpha=0.99, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.476925872093%
dev error: 0.568181818182%
alpha=0.99, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.27707122093%
dev error: 0.532670454545%
alpha=0.99, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.286155523256%
dev error: 0.532670454545%
alpha=0.99, max_ngram_range=6, smooth_idf=F

dev error: 0.532670454545%
alpha=1.0, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.286155523256%
dev error: 0.532670454545%
alpha=1.0, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.458757267442%
dev error: 0.568181818182%
alpha=1.0, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.476925872093%
dev error: 0.568181818182%
alpha=1.0, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.27707122093%
dev error: 0.532670454545%
alpha=1.0, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.286155523256%
dev error: 0.532670454545%
alpha=1.0, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.458757267442%
dev error: 0.568181818182%
alpha=1.0, max_ngram_range=6, smooth_idf=True, 

dev error: 0.568181818182%
alpha=1.0, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.449672965116%
dev error: 0.568181818182%


In [94]:
df_mnb['train_error'] = 100*df_mnb['train_error']

In [95]:
df_mnb.describe()

,alpha,dev_error,max_df,max_ngram_range,min_df,smooth_idf,sublinear_tf,train_error
count,320.000000,320.0,320.000000,320.000000,320.000000,320.000000,320.000000,320.0
mean,0.418220,0.0,0.700000,7.000000,2.000000,0.500000,0.500000,0.0
std,0.473087,0.0,0.200313,1.001566,1.001566,0.500783,0.500783,0.0
min,0.000100,0.0,0.500000,6.000000,1.000000,0.000000,0.000000,0.0
25%,0.001000,0.0,0.500000,6.000000,1.000000,0.000000,0.000000,0.0
50%,0.100000,0.0,0.700000,7.000000,2.000000,0.500000,0.500000,0.0
75%,0.990000,0.0,0.900000,8.000000,3.000000,1.000000,1.000000,0.0
max,1.000000,0.0,0.900000,8.000000,3.000000,1.000000,1.000000,0.0


In [27]:
results = results.split('\n')

In [72]:
import pandas as pd
res = pd.DataFrame(columns=params.keys() + ['train_error_pct', 'dev_error_pct'])
for i, result in enumerate(results):
    if result.startswith('alpha'):
        vals = { k[0]:k[1] for k in map(lambda x: x.split('='), result.split(', '))}
        err = results[i + 1].split(':')[1].rstrip('%')
        vals.update(train_error_pct=float(err))
        err = results[i + 2].split(':')[1].rstrip('%')
        vals.update(dev_error_pct=float(err))
        res = res.append(vals, ignore_index=True)
print(res)                

    smooth_idf max_ngram_range max_df min_df   alpha sublinear_tf  norm  \
0         True               4    0.5      1  0.0001         True    l2   
1         True               4    0.7      1  0.0001         True    l2   
2         True               4    0.9      1  0.0001         True    l2   
3         True               4    0.5      2  0.0001         True    l2   
4         True               4    0.7      2  0.0001         True    l2   
5         True               4    0.9      2  0.0001         True    l2   
6         True               4    0.5      3  0.0001         True    l2   
7         True               4    0.7      3  0.0001         True    l2   
8         True               4    0.9      3  0.0001         True    l2   
9         True               4    0.5      1  0.0001        False    l2   
10        True               4    0.7      1  0.0001        False    l2   
11        True               4    0.9      1  0.0001        False    l2   
12        True           

In [73]:
res.describe()

,train_error_pct,dev_error_pct
count,216.000000,216.000000
mean,0.024561,0.323548
std,0.015784,0.035373
min,0.004542,0.284091
25%,0.013626,0.284091
50%,0.022711,0.319602
75%,0.045422,0.355114
max,0.049964,0.390625


In [74]:
res['train_error_pct'].min()

0.0045421511627899997

In [75]:
res.groupby(by='train_error_pct').describe()

dev_error_pct                                          \
                        count      mean       std       min       25%   
train_error_pct                                                         
0.004542                 48.0  0.301847  0.017944  0.284091  0.284091   
0.013626                 24.0  0.355114  0.000000  0.355114  0.355114   
0.018169                 24.0  0.284091  0.000000  0.284091  0.284091   
0.022711                 48.0  0.319602  0.035887  0.284091  0.284091   
0.040879                 16.0  0.319602  0.000000  0.319602  0.319602   
0.045422                 48.0  0.343277  0.033835  0.319602  0.319602   
0.049964                  8.0  0.390625  0.000000  0.390625  0.390625   

                                               
                      50%       75%       max  
train_error_pct                                
0.004542         0.301847  0.319602  0.319602  
0.013626         0.355114  0.355114  0.355114  
0.018169         0.284091  0.284091  0.284091  
0.022711         0.319602  0.355114  0.355114  
0.040879         0.319602  0.319602  0.319602  
0.045422         0.319602  0.390625  0.390625  
0.049964         0.390625  0.390625  0.390625

In [76]:
res.loc[res['train_error_pct'] == 0.00454215116279]

,smooth_idf,max_ngram_range,max_df,min_df,alpha,sublinear_tf,norm,train_error_pct,dev_error_pct
72,True,6,0.5,1,0.0001,True,l2,0.004542,0.319602
73,True,6,0.7,1,0.0001,True,l2,0.004542,0.319602
74,True,6,0.9,1,0.0001,True,l2,0.004542,0.319602
81,True,6,0.5,1,0.0001,False,l2,0.004542,0.319602
82,True,6,0.7,1,0.0001,False,l2,0.004542,0.319602
83,True,6,0.9,1,0.0001,False,l2,0.004542,0.319602
90,True,6,0.5,1,0.0001,True,None,0.004542,0.319602
91,True,6,0.7,1,0.0001,True,None,0.004542,0.319602
92,True,6,0.9,1,0.0001,True,None,0.004542,0.319602
99,True,6,0.5,1,0.0001,False,None,0.004542,0.319602


In [66]:
type(res['train_error_pct'][0])

str

In [37]:
 map(lambda x: x.split('='), st.split(', '))

[['alpha', '0.0001'],
 ['max_ngram_range', '8'],
 ['smooth_idf', 'False'],
 ['norm', 'None'],
 ['sublinear_tf', 'False'],
 ['min_df', '3'],
 ['max_df', '0.9']]

In [58]:
for alpha in params['alpha']:
    print(alpha)

0.0001
0.001
0.1
0.99
1.0


In [77]:
len(results)

650

In [26]:
results = """
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.0136264534884%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.355113636364%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=4, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0499636627907%
dev error: 0.390625%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.022710755814%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=6, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=True, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=l2, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=True, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.5
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.7
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=1, max_df=0.9
train error: 0.00454215116279%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.5
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.7
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=2, max_df=0.9
train error: 0.0181686046512%
dev error: 0.284090909091%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.5
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.7
train error: 0.0408793604651%
dev error: 0.319602272727%
alpha=0.0001, max_ngram_range=8, smooth_idf=False, norm=None, sublinear_tf=False, min_df=3, max_df=0.9
train error: 0.0454215116279%
dev error: 0.319602272727%
"""